In [1]:
import os
import numpy as np
import base64
import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash import Dash, html, dcc, callback, Input, Output, State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from PIL import Image
import io
import cv2
import easyocr
from PIL import Image
import io
from dash import dash_table
import re
import pandas as pd
from googletrans import Translator
from collections import defaultdict
import pandas as pd
df1=pd.DataFrame()
df2 = pd.DataFrame()


C:\Users\chamseddine.benbouzi\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Utiliser un thème Bootstrap
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],suppress_callback_exceptions=True)

# Créer les éléments de l'interface utilisateur
navbare = dbc.NavbarSimple(
    [
        html.Span(style={"border-left": "1px solid white", "height": "50px", "margin-right": "10px"}),
        html.A(html.Img(src="https://www.ngtrend.com/sites/default/files/styles/large/public/2017-05/astree_0.png?itok=xiuQOx6q", height="50px"), href="http://www.astree.com.tn/", target="_blank"),
        html.Span(style={"border-right": "1px solid white", "height": "50px", "margin-left": "10px"})
        
    ],
    brand=html.Span(" OCR SCANNER ", style={"border": "1px solid white", "padding": "8px"}),
    brand_style={"position": "absolute", "transform": "translateX(-50%)","margin-right": "1500px","margin-left": "0px"},
    color="primary",
    dark=True
)

from dash import callback_context

# Créez un état pour contrôler l'état de l'effondrement des sous-options
data_collapse = html.Div(
    dbc.Collapse(
        [   html.Hr(),
            dbc.NavItem(dbc.NavLink("Personne physique", href="/pf", className="list-group-item-action   fs-6 py-2"),
                        className="list-group-item-action   fs-6 py-2"),
             html.Hr(),
            dbc.NavItem(dbc.NavLink("Personne morale", href="/pm", className="list-group-item-action   fs-6 py-2"),
                        className="list-group-item-action  fs-6 py-2"),
             html.Hr(),
        ],
        id="data-collapse",
    )
)

# Définissez une fonction de rappel pour contrôler l'état de l'effondrement des sous-options
@app.callback(
    Output("data-collapse", "is_open"),
    [Input("data", "n_clicks")],
    [State("data-collapse", "is_open")],
)
def toggle_collapse(n, is_open):
    if n:
        return not is_open
    return False

# Créez votre barre de navigation
nav = dbc.Col(
    dbc.Nav(
        [
            dbc.NavItem(dbc.NavLink("Nouveau client", href="/client", className="list-group-item list-group-item-action  fs-5 py-2")),
             html.Hr(),
            dbc.NavItem(dbc.NavLink("Scanner en bloc", href="/scanner", className="list-group-item list-group-item-action  fs-5 py-2")),
             html.Hr(),
            dbc.NavItem(
                dbc.NavLink("Data", id="data", href="/data", className="list-group-item list-group-item-action  fs-5 py-2"),
                className="list-group-item list-group-item-action  fs-5 py-2"
            ),
             
            data_collapse  # Ajoutez l'élément de collapse pour les sous-options
        ],
        vertical=True,
        pills=True,
        style={"width": "250px", "background-color": "#D6EAF8", "padding": "20px", "height": "2000px",
               "position": "absolute"}
    )
)


layout_client= dbc.Container([
    html.Br(),
        dcc.Dropdown(
            id='select-option',
            options=[
                {'label': 'Choisir le type de personne', 'value': 'd1'},
                {'label': 'CIN', 'value': 'option1'},
                {'label': 'RNE', 'value': 'option2'}
            ],
            value='d1',
            style={'width': '500px'}
        ),
        dcc.Upload(
            id="upload-data",
            multiple=True,
            children=dbc.Button("Sélectionner une image",id='sel', color="primary", className="mx-auto mb-3",
                                 style={"margin-top": "20px"}),
        ),
        dbc.Row([
            dbc.Col(html.Img(id="uploaded-image", className="mx-auto mb-3",
                             style={"width": "80%", "height": "auto"}), width=4),
            dbc.Col(html.Img(id="uploaded-image-1", className="mx-auto mb-3",
                             style={"width": "80%", "height": "auto"}), width=4)]),
        dbc.Col([
            dcc.Loading(
                id="loading-ocr",
                type="default",
                children=[
                    dbc.Button("Exécuter OCR", id="btn-ocr", color="success", className="mx-auto mb-3"),
                    dbc.Table(id="ocr-results", bordered=True, striped=True),
                    dbc.Button("Ajouter client", id="ajout", color="success", className="mx-auto mb-3",style={"display":"none"}),
                    html.Div(id='alert-div'),
                ]
            ),
        ], width=6),
            
        # Placeholder for displaying data layout
        dbc.Container(id='container-content',
                      style={"background-color": "#f0f0f0", "min-width": "200vh", "min-height": "200vh",
                             "position": "auto", "margin-left": "200px", "margin-top": "5px"})
    ], id='out',
        style={"background-color": "#f0f0f0", "min-width": "2000px", "min-height": "2000px", "position": "auto",
               "margin-left": "250px"})

# Attribution du layout initial à l'application
app.layout = html.Div([
    dcc.Location(id='url', refresh=True),
    navbare,
    nav,
    html.Div(id='page-content'),
])

# Création du layout initial pour l'option "Data"
layout_data = html.Div([
     
    dbc.Table.from_dataframe(df1,striped=True,bordered=True,hover=True,responsive=True),
    html.A(
        dbc.Button("Télécharger Excel", color="primary"),
        href="",
        id="download-link",
        download="data.xlsx",
        target="_blank"
    ),
        dbc.Button("Reset", color="danger", id="reset-button",style={"margin-left": "50px"})
], style={"margin": "20px","margin-left": "200px"})  # Ajouter une marge autour du tablea




# Layout pour le scanner OCR
layout_scanner = dbc.Container([
     html.Br(),
    dcc.Dropdown(
        id='select-option1',
        options=[
            {'label': 'Choisir le type de personne', 'value': 'd1'},
            {'label': 'CIN', 'value': 'option1'},
            {'label': 'RNE', 'value': 'option2'}
        ],
        value='d1',
        style={'width': '500px'}
    ),
    dcc.Upload(
        id="upload-data1",
        multiple=True,
        children=dbc.Button("Sélectionner des fichiers",id="sel1", color="primary", className="mx-auto mb-3",
                             style={"margin-top": "20px"}),
    ),
    dbc.Row([ 
        html.Div(id="photo")
    ]),
    dbc.Col([
        dcc.Loading(
            id="loading-ocr1",
            type="default",
            children=[
                dbc.Button("Exécuter OCR", id="btn-ocr1", color="success", className="mx-auto mb-3"),
                dbc.Table(id="ocr-results1", bordered=True, striped=True),
                dbc.Button("Ajouter clients", id="ajout1", color="success", className="mx-auto mb-3",style={"display":"none"}),
                html.Div(id='alert-div1'),
                
            ]
        ),
    ], width=6),
    
    # Placeholder pour l'affichage du layout des données
    dbc.Container(id='container-content1',
                  style={"background-color": "#f0f0f0", "min-width": "200vh", "min-height": "200vh",
                         "position": "auto", "margin-left": "200px", "margin-top": "5px"})
], id='out1',
    style={"background-color": "#f0f0f0", "min-width": "2000px", "min-height": "2000px", "position": "auto",
           "margin-left": "250px","width": "2000px", "height": "2000px"})


# Ajoutez ce callback à votre application Dash
@app.callback(
    Output("photo", "children"),
    [Input("upload-data1", "filename")]
)
def update_photo_filenames(filenames):
    if filenames is not None:
        return [html.P(filename) for filename in filenames]
    else:
        return []
    


# Callback pour afficher le contenu de la page en fonction de l'URL actuelle
@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)
def display_layout(pathname):
    if pathname == '/client':
        return layout_client
    elif pathname == '/scanner':
        return layout_scanner
    elif pathname == '/pf':
        data=pd.read_excel('ocr_results45.xlsx')
        return html.Div([
          html.A(
        dbc.Button("Télécharger Excel", color="primary"),
        href="",
        id="download-link",
        download="data.xlsx",
        target="_blank", 
        style={"margin-left": "250px"}
        
    ),
        dbc.Button("Reset", color="danger", id="reset-button",style={"margin-left": "50px"}),html.Br(),html.Br(),
    dbc.Table.from_dataframe(data,id='table',striped=True,bordered=True,hover=True,responsive=True, style={"margin-left": "250px","width":"78%"})
               
], style={"margin": "20px"})  # Ajouter une marge autour du tablea
    
    
    elif pathname == '/pm':
        data1=pd.read_excel('ocr.xlsx')

        return html.Div([
          html.A(
        dbc.Button("Télécharger Excel", color="primary"),
        href="",
        id="download-link1",
        download="data.xlsx",
        target="_blank", 
        style={"margin-left": "250px"}
        
    ),
        dbc.Button("Reset", color="danger", id="reset-button1",style={"margin-left": "50px"}),html.Br(),html.Br(),
    dbc.Table.from_dataframe(data1,id='table1',striped=True,bordered=True,hover=True,responsive=True, style={"margin-left": "250px","width":"78%"})
               
], style={"margin": "20px"})  # Ajouter une marge autour du tablea
    else:
        return html.Div()


@app.callback(
    Output('table', 'children'),
    [Input('reset-button', 'n_clicks')]
)
def delete_rows(n_clicks):
    data=pd.read_excel('ocr_results45.xlsx')
    if n_clicks is not None:
        # Supprimer toutes les lignes du DataFrame
        data.drop(data.index, inplace=True)
        data.to_excel('ocr_results45.xlsx', index=False)
    # Recréer le tableau à partir du DataFrame mis à jour
    table = dbc.Table.from_dataframe(data, striped=True, bordered=True, hover=True, responsive=True)
    
    return table

@app.callback(
    Output('table1', 'children'),
    [Input('reset-button1', 'n_clicks')]
)
def delete_row1(n_clicks):
    data1=pd.read_excel('ocr.xlsx')
    if n_clicks is not None:
        # Supprimer toutes les lignes du DataFrame
        data1.drop(data1.index, inplace=True)
        data1.to_excel('ocr.xlsx', index=False)
        # Recréer le tableau à partir du DataFrame mis à jour
    table1= dbc.Table.from_dataframe(data1, striped=True, bordered=True, hover=True, responsive=True)
    
    
    
    return table1
    
    
@app.callback(
    Output('download-link', 'href'),
    [Input('download-link', 'n_clicks')]
)
def download_excel(n_clicks):
    if n_clicks:
        # Charger le contenu du fichier Excel
        excel_data = open('ocr_results45.xlsx', 'rb').read()
        excel_base64 = base64.b64encode(excel_data).decode('utf-8')
        href = f'data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{excel_base64}'
        return href
    
@app.callback(
    Output('download-link1', 'href'),
    [Input('download-link1', 'n_clicks')]
)
def download_excel(n_clicks):
    if n_clicks:
        # Charger le contenu du fichier Excel
        excel_data = open('ocr.xlsx', 'rb').read()
        excel_base64 = base64.b64encode(excel_data).decode('utf-8')
        href = f'data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{excel_base64}'
        return href
    
def traduire_texte(nom_arabe):
    correspondance = {
        'ع': 'a',
        'ا': 'a',
        'ب': 'b',
        'ت': 't',
        'ث': 'th',
        'ج': 'j',
        'ح': 'h',
        'خ': 'kh',
        'د': 'd',
        'ذ': 'dh',
        'ر': 'r',
        'ز': 'z',
        'س': 's',
        'ش': 'ch',
        'ص': 's',
        'ض': 'd',
        'ط': 't',
        'ظ': 'z',
        'ع': 'a',
        'غ': 'gh',
        'ف': 'f',
        'ق': 'q',
        'ك': 'k',
        'ل': 'l',
        'م': 'm',
        'ن': 'n',
        'ه': 'h',
        'و': 'ou',
        'ى': 'y',
        'ى': 'i',
        'ي': 'i',
        'ء':'',
        'ة':'a '
        
    }

    # Ajoutez des combinaisons supplémentaires si nécessaire
    combinaisons_supplementaires = {
        'تش': 'tch',
        'ش': 'ch',
        'ج': 'dj',
        'س': 'ss',
        # Ajoutez d'autres combinaisons au besoin
    }

    texte_traduit = ''
    index = 0

    while index < len(nom_arabe):
        lettre = nom_arabe[index]

        # Vérifiez les combinaisons spéciales
        for combinaison, traduction in combinaisons_supplementaires.items():
            if nom_arabe[index:index + len(combinaison)] == combinaison:
                texte_traduit += traduction
                index += len(combinaison)
                break
        else:
            # Utilisez la correspondance normale si aucune combinaison spéciale n'est trouvée
            texte_traduit += correspondance.get(lettre, lettre)
            index += 1

    return texte_traduit



def traduire_profession(nom_propre, src_lang='ar', dest_lang='fr'):
    translator = Translator()

    # Traduire le nom propre
    traduction = translator.translate(nom_propre, src=src_lang, dest=dest_lang)

    return traduction.text





def extract_dos(chaine_complete):
    
   
     # Trouver l'index de "اللقب" et l'index de la première occurrence de "ال" après "اللقب"
    index_debut = chaine_complete.find("هنة")
    index_fin = chaine_complete.find(" ", index_debut + len(" هنة"))

    # Extraire la chaîne entre les deux indices
    chaine_extraite = chaine_complete[index_debut + len("هنة"):index_fin].strip()
    
    # Trouver l'index de "اللقب"
    index_debut = chaine_complete.find("هنة")

    # Trouver l'index de la première occurrence de "ال" après "اللقب"
    index_apres_al = chaine_complete.find(" ", index_debut + len("هنة"))

    # Trouver le reste du mot après "ال"
    reste_mot_apres_al = chaine_complete[index_apres_al + len(" "):].split()[0].strip()

    # Trouver l'index du début du mot suivant après "ال"
    index_debut_mot_suivant = index_apres_al + len(" ") + len(reste_mot_apres_al) + 1

    # Trouver l'index du prochain espace après le début du mot suivant
    index_fin_mot_suivant = chaine_complete.find(" ", index_debut_mot_suivant)

    # Extraire la chaîne entre les deux indices pour le mot suivant
    mot_suivant = chaine_complete[index_debut_mot_suivant:index_fin_mot_suivant].strip()
     # Utiliser une expression régulière pour trouver tous les nombres dans la chaîne
    nombres = re.findall(r'\b\d+\b', chaine_complete)
    # Utiliser une expression régulière pour trouver un mois en arabe dans la chaîne
    mois_arabe = re.search(r'(?:جانفي|فيفري|مارس|افريل|ماي|جوان|جويلية|اوت|سبتمبر|اكتوبر|نوفمبر|ديسمبر)', chaine_complete)
    # Extraire le mois si trouvé
    if mois_arabe:
        mois_arabe = mois_arabe.group()
    else:
        mois_arabe = ""
    
    # Convertir les nombres de chaînes en entiers si nécessaire
    nombres_entiers = [nombre for nombre in nombres]
    nombres_extraits_tries = sorted(nombres_entiers, key=lambda x: len(str(x)))
    nombres_extraits_tries.append(mois_arabe)
    nombres_extraits_tries.append(mot_suivant)
    nombres_extraits_tries.append(chaine_extraite)
    return nombres_extraits_tries
# Fonction pour exécuter l'OCR sur une image spécifiée
def run_ocr_dos(image_path):
    # Création d'un lecteur OCR pour la langue arabe
    reader = easyocr.Reader(['ar'], gpu=True)
    
    # Chargement de l'image
    results = reader.readtext(image_path, detail=1)  # Use detail=1 to get bounding boxes

    # Extracting the text and bounding boxes
    columns = {}

    for result in results:
        text = result[1]
        bbox = result[0]
        current_x = bbox[0][0]

        # If the X-coordinate is not already in columns, create a new entry
        if current_x not in columns:
            columns[current_x] = {"Text": text, "Initial Position": bbox[2][1]}
        else:
            # Extend the current column
            columns[current_x]["Text"] += " "+text + " "

    # Sort the columns based on the X-coordinate
    sorted_columns = sorted(columns.values(), key=lambda x: x["Initial Position"])

    # Extract the text from sorted columns
    extracted_data = [column["Text"] for column in sorted_columns]
    # Concaténation avec un espace comme délimiteur
    extracted_data = " ".join(extracted_data)
    i=extracted_data.find("٢")
    extracted_data=extracted_data[:i-1].strip()
    s=extract_dos(extracted_data)
    # Dictionnaire de traduction des mois
    traduction_mois = {
        'جانفي': 'Janvier',
        'فيفري': 'Février',
        'مارس': 'Mars',
        'افريل': 'Avril',
        'ماي': 'Mai',
        'جوان': 'Juin',
        'جويلية': 'Juillet',
        'اوت': 'Août',
        'سبتمبر': 'Septembre',
        'اكتوبر': 'Octobre',
        'نوفمبر': 'Novembre',
        'ديسمبر': 'Décembre'
    }

    # Traduire le mois de l'arabe vers le français
    mois_francais = traduction_mois.get(s[-3], s[-3])
    # Concaténer les éléments dans l'ordre souhaité avec un espace entre eux
    resultat = s[-5] + ' ' + mois_francais + ' ' + s[-4] 
   
    
    x={"Profession ":traduire_profession(s[-1]),"Date de délivrance":resultat ,}
    return x



import re

def extract(chaine_complete):
    # Trouver l'index de "اللقب"
    index_debut = chaine_complete.find("اللقب")

    # Trouver l'index de la première occurrence de "ال" après "اللقب"
    index_fin = chaine_complete.find("ا", index_debut + len("اللقب"))
    
    # Vérifier si le mot juste après "اللقب" commence par "ال"
    mot_suivant = chaine_complete[index_fin + len("ا"):].split()[0].strip()
    # Trouver le reste du mot après "ال"
    mot_suivant1 = chaine_complete[index_debut + len("اللقب"):].split()[0].strip()
    if mot_suivant.startswith("ال"):
        index_fin = chaine_complete.find("ا", index_debut + len("اللقب"))
    else:
        index_fin = chaine_complete.find("ا",index_debut + len("اللقب")+len(mot_suivant))
        
        
    if "ا" in mot_suivant1:
        # Faites quelque chose si "ا" est présent dans mot_suivant
        index_fin = chaine_complete.find("ا", index_debut + len("اللقب")+ len(mot_suivant1))
    else:
        # Faites quelque chose si "ا" n'est pas présent dans mot_suivant
        index_fin = chaine_complete.find("ا", index_debut + len("اللقب") )
        
    chaine_extraite = chaine_complete[index_debut + len("اللقب"):index_fin].strip()



    # Trouver l'index de "اللقب"
    index_debut = chaine_complete.find("اللقب")

    # Trouver l'index de la première occurrence de "ال" après "اللقب"
    index_apres_al = chaine_complete.find("ا", index_fin )

    # Trouver le reste du mot après "ال"
    reste_mot_apres_al = chaine_complete[index_apres_al + len("ا"):].split()[0].strip()

    # Trouver l'index du début du mot suivant après "ال"
    index_debut_mot_suivant = index_apres_al + len("ا") + len(reste_mot_apres_al) + 1

    # Trouver l'index du prochain espace après le début du mot suivant
    index_fin_mot_suivant = chaine_complete.find(" ", index_debut_mot_suivant)

    # Extraire la chaîne entre les deux indices pour le mot suivant
    mot_suivant = chaine_complete[index_debut_mot_suivant:index_fin_mot_suivant].strip()
     # Utiliser une expression régulière pour trouver tous les nombres dans la chaîne
    nombres = re.findall(r'\b\d+\b', chaine_complete)
    # Utiliser une expression régulière pour trouver un mois en arabe dans la chaîne
    mois_arabe = re.search(r'(?:جانفي|فيفري|مارس|افريل|ماي|جوان|جويلية|اوت|سبتمبر|اكتوبر|نوفمبر|ديسمبر)', chaine_complete)
    # Extraire le mois si trouvé
    if mois_arabe:
        mois_arabe = mois_arabe.group()
    else:
        mois_arabe = ""
    
    # Convertir les nombres de chaînes en entiers si nécessaire
    nombres_entiers = [nombre for nombre in nombres]
    nombres_extraits_tries = sorted(nombres_entiers, key=lambda x: len(str(x)))
    nombres_extraits_tries.append(mois_arabe)
    nombres_extraits_tries.append(mot_suivant)
    nombres_extraits_tries.append(chaine_extraite)
    print(mot_suivant)
    return nombres_extraits_tries







# Fonction pour exécuter l'OCR sur une image spécifiée
def run_ocr(image_path,image_path1):
    # Création d'un lecteur OCR pour la langue arabe
    reader = easyocr.Reader(['ar'], gpu=True)
    
    # Chargement de l'image
    results = reader.readtext(image_path, detail=1)  # Use detail=1 to get bounding boxes

    # Extracting the text and bounding boxes
    columns = {}

    for result in results:
        text = result[1]
        bbox = result[0]
        current_x = bbox[0][0]

        # If the X-coordinate is not already in columns, create a new entry
        if current_x not in columns:
            columns[current_x] = {"Text": text, "Initial Position": bbox[2][1]}
        else:
            # Extend the current column
            columns[current_x]["Text"] += " "+text + " "

    # Sort the columns based on the X-coordinate
    sorted_columns = sorted(columns.values(), key=lambda x: x["Initial Position"])

    # Extract the text from sorted columns
    extracted_data = [column["Text"] for column in sorted_columns]
    # Concaténation avec un espace comme délimiteur
    extracted_data = " ".join(extracted_data)
    s=extract(extracted_data)
    # Dictionnaire de traduction des mois
    traduction_mois = {
        'جانفي': 'Janvier',
        'فيفري': 'Février',
        'مارس': 'Mars',
        'افريل': 'Avril',
        'ماي': 'Mai',
        'جوان': 'Juin',
        'جويلية': 'Juillet',
        'اوت': 'Août',
        'سبتمبر': 'Septembre',
        'اكتوبر': 'Octobre',
        'نوفمبر': 'Novembre',
        'ديسمبر': 'Décembre'
    }

    # Traduire le mois de l'arabe vers le français
    mois_francais = traduction_mois.get(s[3], s[3])
    # Concaténer les éléments dans l'ordre souhaité avec un espace entre eux
    resultat = s[0] + ' ' + mois_francais + ' ' + s[1]
   
    x1=run_ocr_dos(image_path1)
    x={"Prénom":traduire_texte(s[-2]).upper(),"Nom":traduire_texte(s[-1]).upper(),"Numéro de CIN":s[-4],"Date de naissance":resultat}
    x.update(x1)
    
    return x




# Fonction pour détecter les tables dans une image
def detecter_tables(image):
    # Convertir l'image en niveaux de gris
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Appliquer un seuillage adaptatif pour améliorer la visibilité des chiffres
    _, seuil = cv2.threshold(gris, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Détecter les contours dans l'image seuillée
    contours, _ = cv2.findContours(seuil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    tables = []

    # Filtrer les contours en fonction de l'aspect ratio
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        # Filtrer les contours avec un aspect ratio proche de celui d'un rectangle
        aspect_ratio = w / float(h)
        if 0.5 < aspect_ratio < 2.0:
            tables.append((x, y, w, h))
    
    return tables




# Fonction pour détecter les tables dans une image et appliquer l'OCR
def appliquer_ocr_sur_tables(image_path):
    # Charger l'image
    image = cv2.imread(image_path)
    # Détecter les tables dans l'image
    tables_detectees = detecter_tables(image)

    # Afficher l'image avec les tables détectées
    image_tables = image.copy()
    for table in tables_detectees:
        x, y, w, h = table
        cv2.rectangle(image_tables, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Appliquer l'OCR sur les tables détectées
    reader = easyocr.Reader(['fr'], gpu=True)  # Assurez-vous que le paramètre GPU est désactivé
    for table in tables_detectees:
        x, y, w, h = table
        table_roi = image[y:y + h, x:x + w]

    # Appliquer l'OCR sur chaque table
    resultats_ocr_par_table = reader.readtext(table_roi, detail=0)
    return resultats_ocr_par_table


def RNE(image_path):
    s = appliquer_ocr_sur_tables(image_path)
    # Default value for results
    results = None
    
    for i in range(len(s)):
        if 'Ident' in s[i]:
            results = s[i + 1]

    # Check if results is still None
    if results is None:
        raise ValueError("Identification information not found in OCR results.")

    # Modify the results string
    modified_results = results[:-1].replace('o', '0').replace('O', '0') + results[-1].upper()

    # Return the modified information as a dictionary
    results1 = {"Numéro de RNE": modified_results}
    return results1





# Fonction pour détecter les tables dans une image et appliquer l'OCR
def appliquer_ocr_sur_tables1(image):
    # Détecter les tables dans l'image
    tables_detectees = detecter_tables(image)

    # Initialiser le lecteur OCR
    reader = easyocr.Reader(['fr'], gpu=True)  # Assurez-vous que le paramètre GPU est désactivé

    resultats_ocr_par_table = []
    # Appliquer l'OCR sur chaque table détectée
    for table in tables_detectees:
        x, y, w, h = table
        table_roi = image[y:y + h, x:x + w]

        # Appliquer l'OCR sur la table
        results = reader.readtext(table_roi, detail=0)
        resultats_ocr_par_table.extend(results)

    return resultats_ocr_par_table


def RNE1(image_path):
    # Default value for results
    results = None
    # Décodez l'image à partir de base64
    nparr = np.frombuffer(base64.b64decode(image_path), np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    # Appliquer l'OCR sur les tables dans l'image
    s = appliquer_ocr_sur_tables1(image)
    
    for i in range(len(s)):
        if 'Ident' in s[i]:
            results = s[i + 1]

    # Check if results is still None
    if results is None:
        raise ValueError("Identification information not found in OCR results.")

    # Modify the results string
    modified_results = results[:-1].replace('o', '0').replace('O', '0') + results[-1].upper()

    # Return the modified information as a dictionary
    results11 = {"Numéro de RNE": modified_results}
    return results11






@app.callback(
    [Output("uploaded-image", "src"),
     Output("uploaded-image-1", "src")],
    [Input("upload-data", "contents")],
    [State('select-option', 'value')]
)
def update_uploaded_images(contents, selected_option):
    if contents is None or len(contents) < 1:
        raise PreventUpdate
    
    # Si une seule image est sélectionnée
    if len(contents) == 1:
        _, content_string = contents[0].split(",")
        image_data = base64.b64decode(content_string)

        # Save the image in a temporary file
        image_path = "temp_image.jpg"

        with open(image_path, "wb") as f:
            f.write(image_data)

        # Display the image
        image_source = f"data:image/jpeg;base64,{content_string}"

        return image_source, None

    # Si deux images sont sélectionnées
    elif len(contents) == 2:
        image_sources = []

        for i, content in enumerate(contents[:2]):
            _, content_string = content.split(",")
            image_data = base64.b64decode(content_string)

            # Save the image in a temporary file
            image_path = f"temp_image_{i+1}.jpg"

            with open(image_path, "wb") as f:
                f.write(image_data)

            # Display the image
            image_sources.append(f"data:image/jpeg;base64,{content_string}")

        return tuple(image_sources)






    

    
    
    



    
    


@app.callback(
    Output("ocr-results", "children"),
    Output("ajout", "style"),
    Input("btn-ocr", "n_clicks"),
    State("uploaded-image", "src"),
    State("uploaded-image-1", "src"),
    State('select-option', 'value'),
    prevent_initial_call=True# Ajouter la valeur de l'élément RadioItems comme entrée
)
def execute_ocr(n_clicks, src, src1, selected_option):
    global df22 
    if  src is None:
        raise PreventUpdate
    elif src1 is None:
       
        # Extract the type of the image from the source
        image_type = src.split(";")[0].split("/")[-1]

        # Create a temporary image path
        temp_image_path = "temp_image." + image_type

        # Save the updated image with the correct path
        with open(temp_image_path, "wb") as f:
            f.write(base64.b64decode(src.split(",")[1]))

        # Exécuter l'OCR en fonction du type d'image et de l'option sélectionnée
        if image_type in ["jpeg", "png"]:
            if selected_option == 'option2':
                # Utiliser la fonction appliquer_ocr_sur_tables
                results = RNE(temp_image_path)
                # Transposer le dictionnaire pour avoir les clés en tant qu'en-têtes de tableau et les valeurs en tant que ligne
                transposed_results = {key: [value] for key, value in results.items()}


                df = pd.DataFrame(transposed_results)
                df.to_excel('ocr_res1.xlsx', index=False)
        else:
            raise ValueError("Format d'image non pris en charge")


        # Construire la table des résultats
        table = dbc.Table.from_dataframe(df, bordered=True, striped=True, hover=True)
        style={"display":"inline"}
        return table,style


    elif src is not None and  src1 is not None:
        # Extract the type of the image from the source
        image_type = src.split(";")[0].split("/")[-1]

        # Create a temporary image path
        temp_image_path = "temp_image." + image_type

        # Save the updated image with the correct path
        with open(temp_image_path, "wb") as f:
            f.write(base64.b64decode(src.split(",")[1]))

        if n_clicks is None or src1 is None:
            raise PreventUpdate

        # Extract the type of the image from the source
        image_type1 = src1.split(";")[0].split("/")[-1]

        # Create a temporary image path
        temp_image_path1 = "temp_image1." + image_type1

        # Save the updated image with the correct path
        with open(temp_image_path1, "wb") as f1:
            f1.write(base64.b64decode(src1.split(",")[1]))
       
        # Exécuter l'OCR en fonction du type d'image et de l'option sélectionnée
        if image_type in ["jpeg", "png"]:
            if selected_option == 'option1':
                results = run_ocr(temp_image_path, temp_image_path1)
                # Transposer le dictionnaire pour avoir les clés en tant qu'en-têtes de tableau et les valeurs en tant que ligne
                transposed_results = {key: [value] for key, value in results.items()}

            
                df22 = pd.DataFrame(transposed_results)
                df22.to_excel('ocr_res.xlsx', index=False)
        else:
            raise ValueError("Format d'image non pris en charge")


        # Construire la table des résultats
        table = dbc.Table.from_dataframe(df22, bordered=True, striped=True, hover=True)
        style={"display":"inline"}
        return table,style

from collections import defaultdict
from os.path import basename
# Callback pour exécuter l'OCR
@app.callback(
    Output("ocr-results1", "children"),
    Output("ajout1", "style"),
    Input("btn-ocr1", "n_clicks"),
    State("upload-data1", "filename"),
    State("upload-data1", "contents"),
    State("select-option1", "value")
)

def execute_ocr1(n_clicks, filenames, contents, selected_option):
    if n_clicks is None or filenames is None or contents is None:
        raise PreventUpdate
    
    

    # Vérifier que le nombre de fichiers téléchargés est pair
    
    global df2
    if selected_option == "option1":
        if len(filenames) % 2 != 0:
            raise PreventUpdate
        all_results = []
        # Regrouper les fichiers par nom de client
        client_files = defaultdict(list)
        for filename in filenames:
            client_name, file_type = basename(filename).split('_')
            client_files[client_name].append((filename, file_type))

        # Parcourir les paires de fichiers pour chaque client
        for client_name, files in client_files.items():
            faces = [file for file in files if file[1].endswith(('f.jpg', 'f.png'))]
            backs = [file for file in files if file[1].endswith(('d.jpg','d.png'))]

            # Assurez-vous que chaque face a un dos correspondant
            for face in faces:
                matching_backs = [back for back in backs if back[0].startswith(face[0].split('_')[0])]
                if len(matching_backs) == 1:
                    face_file, back_file = face[0], matching_backs[0][0]
                    # Appliquer l'OCR à la paire de fichiers
                    results = run_ocr(face_file, back_file)  # Remplacez run_ocr par votre fonction OCR
                    all_results.append(results)
                else:
                    print(f"Aucun dos correspondant trouvé pour {face[0]}")

        # Convertir les résultats en DataFrame
        df = pd.DataFrame(all_results)



        # Enregistrer les résultats dans un fichier Excel
        df.to_excel('ocr_res.xlsx', index=False)
            # Construire la table des résultats
        table = dbc.Table.from_dataframe(df, bordered=True, striped=True, hover=True)
        # Afficher les résultats dans une table
        style11={"display":"inline"}
        return table,style11
    elif selected_option == "option2":
        # Récupérer les chemins des fichiers à partir des contenus
        file_paths = [content.split(',')[1] for content in contents]

        all_results = []
        # Traiter toutes les images en bloc
        for file_path in file_paths:
            # Appliquer l'OCR à chaque image
            results = RNE1(file_path)  # Appeler votre fonction RNE avec le chemin du fichier
            all_results.append(results)

        # Convertir les résultats en DataFrame
        df455 = pd.DataFrame(all_results)

        # Enregistrer les résultats dans un fichier Excel
        df455.to_excel('ocr_res1.xlsx', index=False)

        # Construire la table des résultats
        table = dbc.Table.from_dataframe(df455, bordered=True, striped=True, hover=True)

        # Afficher les résultats dans une table
        style11={"display":"inline"}
        return table, style11
    
    

@app.callback(
    Output("ajout", "color"),
    Output('alert-div', 'children'),
    Input("ajout", "n_clicks"),
    State("btn-ocr", "n_clicks"),
    State('select-option', 'value'),
    State("ocr-results", "children")
)
def add_ocr_results_to_df2(n_clicks,n_clicks1,select_option,ocr_result):
    s="secondary"
    alert=""
    if n_clicks and ocr_result :
        if select_option=="option1":
            df5 = pd.read_excel('ocr_results45.xlsx')
            # Charger le DataFrame existant
            df4 = pd.read_excel('ocr_res.xlsx')
            if df5.empty:
                # Ajouter les nouvelles données et écrire dans le fichier Excel
                df5 = df5.append(df4, ignore_index=True)
                df5.to_excel('ocr_results45.xlsx', index=False)
                s="success"
                alert= dbc.Alert("Client ajouté avec succès", color="success")
                
            elif df4["Numéro de CIN"].isin(df5["Numéro de CIN"]).any():
                alert= dbc.Alert("Ce client existe !", color="danger")
                s="danger"
            else :
                # Ajouter les nouvelles données et écrire dans le fichier Excel
                df5 = df5.append(df4, ignore_index=True)
                df5.to_excel('ocr_results45.xlsx', index=False)
                s="success"
                alert= dbc.Alert("Client ajouté avec succès", color="success")
        elif select_option=="option2":
            df10 = pd.read_excel('ocr.xlsx')
            # Charger le DataFrame existant
            df11 = pd.read_excel('ocr_res1.xlsx')
            if df11['Numéro de RNE'].isin(df10['Numéro de RNE']).any():
                alert= dbc.Alert("Ce client existe !", color="danger")
                s="danger"
            else :
                # Ajouter les nouvelles données et écrire dans le fichier Excel
                df10 = df10.append(df11, ignore_index=True)
                df10.to_excel('ocr.xlsx', index=False)
                s="success"
                alert= dbc.Alert("Client ajouté avec succès", color="success")
    # Vérifiez si le bouton "Executer OCR" a été cliqué à nouveau
    
    return s,alert





global df6

@app.callback(
    Output("ajout1", "color"),
    Output('alert-div1', 'children'),
    Input("ajout1", "n_clicks"),
    State('select-option1', 'value'),
    State("ocr-results1", "children")  # Utilisez les données de la table comme State
)
def add_ocr_results_to_df(n_clicks, select_option1, ocr_result):
    s1 = "secondary"
    alert = None

    if n_clicks and ocr_result:
        if select_option1 == "option1":
            # Lire les données existantes du fichier Excel
            df6 = pd.read_excel('ocr_results45.xlsx')
            df4 = pd.read_excel('ocr_res.xlsx')
            
            # Identifier les lignes avec des numéros de CIN en double
            duplicate_rows = df6[df6['Numéro de CIN'].isin(df4['Numéro de CIN'])]
            
            if df6.empty:
                # Garder uniquement les colonnes de df4 lors de la concaténation pour éviter les doublons
                df6 = pd.concat([df6, df4], ignore_index=True)
                df6.to_excel('ocr_results45.xlsx', index=False)
                s1 = "success"
                alert = dbc.Alert("Clients ajoutés avec succès", color="success")
            elif not duplicate_rows.empty:
                # Identifier les lignes de df4 qui ne sont pas en double
                unique_rows_df4 = df4[~df4['Numéro de CIN'].isin(duplicate_rows['Numéro de CIN'])]
                
                # Concaténer uniquement les lignes de df4 qui ne sont pas en double avec df6
                df6 = pd.concat([df6, unique_rows_df4], ignore_index=True)
                df6.to_excel('ocr_results45.xlsx', index=False)
                
                # Extraire les numéros de CIN en double
                duplicate_cins = duplicate_rows['Numéro de CIN'].unique()
                
                # Créer une chaîne contenant les numéros de CIN en double
                cins_str = ", ".join(map(str, duplicate_cins))
                alert_message = f"Certains numéros de CIN ({cins_str}) existent déjà dans la base de données.les autres clients sont ajoutés avec succès"
                alert = dbc.Alert(alert_message, color="warning")
                s1 = "warning"
            else:
                # Garder uniquement les colonnes de df4 lors de la concaténation pour éviter les doublons
                df6 = pd.concat([df6, df4], ignore_index=True)
                df6.to_excel('ocr_results45.xlsx', index=False)
                s1 = "success"
                alert = dbc.Alert("Clients ajoutés avec succès", color="success")

        elif select_option1 == "option2":
            # Lire les données existantes du fichier Excel
            df12 = pd.read_excel('ocr.xlsx')
            df11 = pd.read_excel('ocr_res1.xlsx')

            # Identifier les lignes avec des numéros de RNE en double
            duplicate_rows = df12[df12['Numéro de RNE'].isin(df11['Numéro de RNE'])]

            if df12.empty:
                # Garder uniquement les colonnes de df11 lors de la concaténation pour éviter les doublons
                df12 = pd.concat([df12, df11], ignore_index=True)
                df12.to_excel('ocr.xlsx', index=False)
                s1 = "success"
                alert = dbc.Alert("Clients ajoutés avec succès", color="success")
            elif not duplicate_rows.empty:
                # Identifier les lignes de df11 qui ne sont pas en double
                unique_rows_df11 = df11[~df11['Numéro de RNE'].isin(duplicate_rows['Numéro de RNE'])]
                
                # Concaténer uniquement les lignes de df11 qui ne sont pas en double avec df12
                df12 = pd.concat([df12, unique_rows_df11], ignore_index=True)
                df12.to_excel('ocr.xlsx', index=False)
                
                # Extraire les numéros de RNE en double
                duplicate_rnes = duplicate_rows['Numéro de RNE'].unique()
                
                # Créer une chaîne contenant les numéros de RNE en double
                rne_str = ", ".join(map(str, duplicate_rnes))
                alert_message = f"Certains numéros de RNE ({rne_str}) existent déjà dans la base de données. Les autres clients sont ajoutés avec succès."
                alert = dbc.Alert(alert_message, color="warning")
                s1 = "warning"
            else:
                # Garder uniquement les colonnes de df11 lors de la concaténation pour éviter les doublons
                df12 = pd.concat([df12, df11], ignore_index=True)
                df12.to_excel('ocr.xlsx', index=False)
                s1 = "success"
                alert = dbc.Alert("Clients ajoutés avec succès", color="success")

    return s1, alert


# Définissez une fonction de rappel pour contrôler l'état de l'effondrement des sous-options
@app.callback(
    Output("alert-div", "children",allow_duplicate=True),
    Output("ajout", "color",allow_duplicate=True),
    Output("ocr-results", "children",allow_duplicate=True),
    Output("ajout", "style",allow_duplicate=True),
    [Input('sel', 'n_clicks')],
    prevent_initial_call=True
)
def initialiser(n_clicks):
    if n_clicks:
        alert=""
        s = "secondary"
        df55=pd.DataFrame()
         # Construire la table des résultats
        table = dbc.Table.from_dataframe(df55, bordered=True, striped=True, hover=True)
        style={"display":"none"}                                                                                                      
                                                                                                              
    return alert,s,table,style


@app.callback(
    Output("alert-div1", "children",allow_duplicate=True),
    Output("ajout1", "color",allow_duplicate=True),
    Output("ocr-results1", "children",allow_duplicate=True),
    Output("ajout1", "style",allow_duplicate=True),
    [Input('sel1', 'n_clicks')],
    prevent_initial_call=True
)
def initialiser1(n_clicks):
    if n_clicks:
        alert=""
        s = "secondary"
        df55=pd.DataFrame()
         # Construire la table des résultats
        table = dbc.Table.from_dataframe(df55, bordered=True, striped=True, hover=True)
        style={"display":"none"}                                                                                                      
                                                                                                              
    return alert,s,table,style


# Exécuter l'application
if __name__ == "__main__":
    app.run_server(debug=True, host="127.0.0.1", port=8050)
